# 🌾 CropFresh AI - Agentic RAG System

Welcome to the **CropFresh AI** interactive notebook.

### ⚠️ Recommended: Run in Browser
For the best experience (especially for file uploads), please run this notebook in **Chrome or Edge** at [colab.research.google.com](https://colab.research.google.com), not inside VS Code.

### 🚀 Setup Instructions
1. Run the **Install Dependencies** cell.
2. Choose **Option A (Upload)** OR **Option B (Drive)** to load the code.
3. Enter your **Groq API Key**.
4. Enjoy the AI Assistant!

## 1. Install Dependencies

In [ ]:
# Install core dependencies fast using uv
!pip install uv
!uv pip install "langgraph>=0.2.0" "langchain>=0.3.0" "langchain-community" "groq" "qdrant-client" "sentence-transformers" "loguru" "pydantic" "python-dotenv" "nest_asyncio" --system

## 2. Load Project Code (Choose One Option)
Run **Option A** OR **Option B**. Do not run both.

In [ ]:
# === OPTION A: DIRECT UPLOAD (Recommended for Browser) ===
import os
from google.colab import files

if not os.path.exists("src"):
    print("📤 OPTION A: Upload 'cropfresh_colab_package.zip' below...")
    try:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        print(f"📦 Unzipping {filename}...")
        !unzip -q -o {filename}
        print("✅ Code ready!")
    except Exception as e:
        print(f"⚠️ Upload failed (typical in VS Code). Try Option B.")
else:
    print("✅ Code already loaded.")

In [ ]:
# === OPTION B: GOOGLE DRIVE (Backup) ===
import os
from google.colab import drive

if not os.path.exists("src"):
    print("🔄 OPTION B: Mounting Google Drive... (Approve popup)")
    try:
        drive.mount('/content/drive')
        
        # Check common paths
        possible_paths = [
            '/content/drive/MyDrive/cropfresh_colab_package.zip',
            '/content/drive/MyDrive/notebooks/cropfresh_colab_package.zip',
            '/content/drive/MyDrive/Colab Notebooks/cropfresh_colab_package.zip'
        ]
        
        found = False
        for zip_path in possible_paths:
            if os.path.exists(zip_path):
                print(f"📦 Found package at: {zip_path}")
                !unzip -q -o "{zip_path}"
                print("✅ Code ready!")
                found = True
                break
        
        if not found:
            print("❌ File not found in Drive!")
            print("Please upload 'cropfresh_colab_package.zip' to your Drive Root or 'notebooks' folder.")
            
    except Exception as e:
        print(f"❌ Drive mount failed: {e}")
else:
    print("✅ Code already loaded.")

## 3. Configuration & API Keys

In [ ]:
import os
import getpass

# Set API Key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq API Key: ")

# Set Layout
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.append('.')

print("✅ Configuration set.")

## 4. Initialize & Populate Knowledge Base
This step creates the Qdrant vector database (in-memory) and ingests sample agricultural data.

In [ ]:
from scripts.populate_qdrant import populate_qdrant
from src.config import get_settings

# Force in-memory for Colab ease-of-use
os.environ["QDRANT_HOST"] = ":memory:"

# Run population
await populate_qdrant()

## 5. Start Agent System
Initialize the Supervisor Agent and start the interactive chat loop.

In [ ]:
import asyncio
from src.agents.supervisor_agent import SupervisorAgent
from src.agents.agronomy_agent import AgronomyAgent
from src.agents.commerce_agent import CommerceAgent
from src.agents.platform_agent import PlatformAgent
from src.agents.general_agent import GeneralAgent
from src.memory.state_manager import AgentStateManager
from src.tools.registry import get_tool_registry
from src.rag.knowledge_base import KnowledgeBase
from src.orchestrator.llm_provider import create_llm_provider

async def start_chat():
    print("🔄 Initializing Agent System...")
    
    # 1. Setup Infrastructure
    kb = KnowledgeBase(host=":memory:")
    await kb.initialize()
    
    llm = create_llm_provider(provider="groq", api_key=os.environ["GROQ_API_KEY"])
    state_manager = AgentStateManager()
    tool_registry = get_tool_registry()
    
    # 2. Create Supervisor
    supervisor = SupervisorAgent(
        llm=llm, 
        state_manager=state_manager, 
        tool_registry=tool_registry, 
        knowledge_base=kb
    )
    
    # 3. Register Agents
    agents = {
        "agronomy_agent": AgronomyAgent(llm=llm, state_manager=state_manager, tool_registry=tool_registry, knowledge_base=kb),
        "commerce_agent": CommerceAgent(llm=llm, state_manager=state_manager, tool_registry=tool_registry, knowledge_base=kb),
        "platform_agent": PlatformAgent(llm=llm, state_manager=state_manager, tool_registry=tool_registry, knowledge_base=kb),
        "general_agent": GeneralAgent(llm=llm, state_manager=state_manager, tool_registry=tool_registry, knowledge_base=kb),
    }
    
    for name, agent in agents.items():
        supervisor.register_agent(name, agent)
        await agent.initialize()
        
    supervisor.set_fallback_agent(agents["general_agent"])
    await supervisor.initialize()
    
    print("\n✨ System Ready! Type 'exit' to stop.")
    print("--------------------------------------------------")
    
    # 4. Chat Loop
    while True:
        try:
            query = input("\n👤 You: ")
            if query.lower() in ["exit", "quit"]:
                break
                
            if not query.strip():
                continue
                
            print("🤖 AI processing...")
            response = await supervisor.process(query)
            
            print(f"\n🤖 {response.agent_name.upper()} ({response.confidence:.0%})")
            print(f"📝 {response.content}")
            
            if response.sources:
                print("\n📚 Sources:")
                for source in response.sources[:2]:
                    print(f"   - {source}")
            
            print("--------------------------------------------------")
            
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"❌ Error: {e}")

await start_chat()